# Importing Libraries

In [1]:
import re
import numpy as np
import pandas as pd
import random
import csv
import subprocess
import warnings
import os
warnings.filterwarnings('ignore') 

# 1. Generating files for delay

## 1.1 Functions

### 1.1.1 Function to extract parameter values from the file

In [2]:
# Function to extract parameter value from content
def extract_parameter_value(file_content, parameter):
    matches = re.findall(rf'\b{parameter}\s*=\s*([\d.e+-]+)', file_content)
    if matches:
        return matches[0], float(matches[1])  # Return the second occurrence
    else:
        return None, None

### 1.1.2 Function to calculate mean and standard deviation for parameters


In [3]:
def calculate_mean_std(file_path, parameter):
    with open(file_path, 'r') as file:
        content = file.read()
    _, mean = extract_parameter_value(content, parameter)
    std = mean / 30  # Assuming a standard deviation of 1/30th of the mean
    return mean, std

### 1.1.4 Function to modify files

In [4]:
import re

def modify_file(file_path, temp, supply, cqload, lmin, wmin, whichBlock, no_of_inputs):
    with open(file_path, 'r') as file:
        data = file.read()
        lines = file.readlines()

    data = re.sub(r'^dc TEMP .+$', f'dc TEMP {temp:.2f} {temp:.2f} {85}', data, flags=re.MULTILINE)
    data = re.sub(r'^\.PARAM SUPPLY=.+$', f'.PARAM SUPPLY={supply:.2f}', data, flags=re.MULTILINE)
    data = re.sub(r'Cqload Vout gnd .+f', f'Cqload Vout gnd {cqload:.2f}f', data)
    data = re.sub(r'\.PARAM Lmin=.+$', f'.PARAM Lmin={lmin:.2f}n', data, flags=re.MULTILINE)
    data = re.sub(r'\.PARAM Wmin=.+$', f'.PARAM Wmin={wmin:.2f}n', data, flags=re.MULTILINE)
    
    with open(file_path, 'w') as file:
        file.write(data)

    with open(file_path, 'r') as file:
        lines = file.readlines()

    curBlock_Vin, curBlock_meas = 0, 0
    cnt_Vin, cnt_meas = no_of_inputs, 2
    for i in range(len(lines)):
        if lines[i].startswith('*Vin1'):
            curBlock_Vin += 1
        
        if curBlock_Vin == whichBlock and cnt_Vin >= 1:
            if lines[i].startswith('*Vin1') or lines[i].startswith('*Vin2') or lines[i].startswith('*Vin3') or lines[i].startswith('*Vin4'):
                lines[i] = lines[i][1:]
                cnt_Vin -= 1
        
        if curBlock_meas // 2 == whichBlock - 1 and cnt_meas >= 1:
            if lines[i].startswith('*.measure'):
                lines[i] = lines[i][1:]
                cnt_meas -= 1
        
        if lines[i].startswith('*.measure'):
            curBlock_meas += 1
    
    with open(file_path, 'w') as file:
        file.writelines(lines)


### 1.1.5 Function to modify leakage files

In [5]:
import re

def modify_file_leakage(file_path, temp, supply, lmin, wmin, *Vin_values):
    with open(file_path, 'r') as file:
        data = file.read()
        lines = file.readlines()

    data = re.sub(r'^dc TEMP .+$', f'dc TEMP {temp:.2f} {temp:.2f} {85}', data, flags=re.MULTILINE)
    data = re.sub(r'^\.PARAM SUPPLY=.+$', f'.PARAM SUPPLY={supply:.2f}', data, flags=re.MULTILINE)
    data = re.sub(r'\.PARAM Lmin=.+$', f'.PARAM Lmin={lmin:.2f}n', data, flags=re.MULTILINE)
    data = re.sub(r'\.PARAM Wmin=.+$', f'.PARAM Wmin={wmin:.2f}n', data, flags=re.MULTILINE)
    
    with open(file_path, 'w') as file:
        file.write(data)

    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    j = 0
    for i in range(len(lines)):
        if lines[i].startswith(f'Vin{j+1}'):
            parts = lines[i].split()
            parts[-1] = str(Vin_values[j])
            lines[i] = ' '.join(parts) + '\n'
            j += 1

    with open(file_path, 'w') as file:
        file.writelines(lines)


### 1.1.6 Function to run ngspice commands

In [6]:
import subprocess
import re

def run_ngspice(file_path):
    output = subprocess.check_output(['ngspice', '-b', file_path]).decode('utf-8')
    delay_lh_match = re.search(r'delay_lh\s*=\s*([\d.e+-]+)', output)
    delay_hl_match = re.search(r'delay_hl\s*=\s*([\d.e+-]+)', output)
    delay_lh = float(delay_lh_match.group(1)) if delay_lh_match else None
    delay_hl = float(delay_hl_match.group(1)) if delay_hl_match else None
    
    return delay_lh, delay_hl


### 1.1.7 Function to calculate delay for a file and save it as csv file

Inputs:
1. PTM file (.pm)
2. Circuit Netlist (.net)
3. Number of samples to generate
4. CSV file path

Output:
1. Modified CSV file

In [7]:
import csv
import numpy as np
import pandas as pd

def generate_delay(parameters, ptm_file, netlist_file, csv_file, no_of_inputs, df_samples):
    
    # Load file content
    with open(ptm_file, 'r') as file:
        original_content = file.read()
        
    with open(netlist_file, 'r') as file:
        original_content_net = file.read()
        
    # Make a copy of the original content for modification
    modified_content = original_content

    # Replace parameter values in the original file with sampled values
    with open(csv_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header_row = ['TEMP', 'pvdd', 'Cqload','Lmin', 'Wmin', 'toxe_n', 'toxm_n', 'toxref_n', 'toxp_n', 'xj_n', 'ndep_n','toxe_p', 'toxm_p', 'toxref_p', 'toxp_p', 'xj_p', 'ndep_p']
        for node in range(no_of_inputs):
            header_row += [f'delay_node{chr(97 + node)}']
        num_rows = sum(1 for _ in open(csv_file, 'r'))
        if num_rows==0:
            writer.writerow(header_row)
            
        try: 
            for i in range(len(df_samples)):
                i+=max(0,num_rows-1)
                row = []
                
                # Extracting samples from DataFrame
                lmin = df_samples['lmin'][i]
                wmin = df_samples['wmin'][i]
                temp = df_samples['temp'][i]
                supply = df_samples['supply'][i]
                cqload = df_samples['cqload'][i]
                
                # Replace parameter values in the original file with sampled values
                for param in parameters:
                    nmos_value=df_samples[param+'_n'][i]
                    pmos_value=df_samples[param+'_p'][i]
                    modified_content = re.sub(rf'\b{param}\s*=\s*([\d.e+-]+)', f'{param} = {pmos_value:.6e}', modified_content, count=2)
                    modified_content = re.sub(rf'\b{param}\s*=\s*([\d.e+-]+)', f'{param} = {nmos_value:.6e}', modified_content, count=1)   
                with open(ptm_file, 'w') as file:
                    file.write(modified_content)
                    
                delay_values = []
                for node in range(no_of_inputs):
                    modify_file(netlist_file, temp, supply, cqload, lmin, wmin, node+1, no_of_inputs)
                    
                    delay_lh, delay_hl = run_ngspice(netlist_file)
                    
                    with open(netlist_file, 'w') as file:
                        file.write(original_content_net)
                    
                    delay_values.append((delay_hl+delay_lh)/2)
                
                row.extend([temp, supply, cqload, lmin * 1e-9, wmin * 1e-9])
                row.extend([df_samples[param+'_n'][i] for param in parameters])
                row.extend([df_samples[param+'_p'][i] for param in parameters])
                row.extend(delay_values)
                writer.writerow(row)
    
        finally:
            # Restore original content of ptm_file and netlist_file
            with open(ptm_file, 'w') as file:
                file.write(original_content)
                
            with open(netlist_file, 'w') as file:
                file.write(original_content_net)

            print(f"Samples generated and saved in {csv_file}. Original files restored.")
    


## 1.2 Defining Parameters to search for

In [8]:
# Load parameters from file
parameters = [
    'toxe', 'toxm', 'toxref','toxp', 'xj', 'ndep'
]

In [9]:
df_samples=pd.read_csv('samples_45nm_MGK.csv')

## 1.3 Generating Delay matrices for all files

### 1.3.1 Gates essential for C499 Circuit

1. INVERTER

In [ ]:
directory = 'INVERTER'
file_name = 'INVERTER_delay.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
rows_for_delay=df_samples[max(0,num_rows-1):]

generate_delay(parameters,'INVERTER/45nm_MGK.pm', 'INVERTER/INVERTER.net', 'INVERTER/INVERTER_delay.csv',1,rows_for_delay)

2. AND_2

In [ ]:
directory = 'AND'
file_name = 'AND_2_delay.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
rows_for_delay=df_samples[max(0,num_rows-1):]

generate_delay(parameters,'AND/45nm_MGK.pm', 'AND/AND.net', 'AND/AND_2_delay.csv',2,rows_for_delay)

3. AND_3

In [ ]:
directory = 'AND_3'
file_name = 'AND_3_delay.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
rows_for_delay=df_samples[max(0,num_rows-1):]

generate_delay(parameters,'AND_3/45nm_MGK.pm', 'AND_3/AND.net', 'AND_3/AND_3_delay.csv',3,rows_for_delay)

Note: Starting dynamic gmin stepping
Trying gmin =   1.0000E-03 Note: One successful gmin step
Trying gmin =   1.0000E-04 Note: One successful gmin step
Trying gmin =   1.0000E-05 Note: One successful gmin step
Trying gmin =   1.0000E-06 Note: One successful gmin step
Trying gmin =   1.0000E-07 Note: One successful gmin step
Trying gmin =   1.0000E-08 Note: One successful gmin step
Trying gmin =   1.0000E-09 Note: One successful gmin step
Trying gmin =   1.0000E-10 Note: One successful gmin step
Trying gmin =   1.0000E-11 Note: One successful gmin step
Trying gmin =   1.0000E-12 Note: One successful gmin step
Trying gmin =   1.0000E-13 Note: One successful gmin step
Trying gmin =   1.0000E-14 Note: One successful gmin step
Trying gmin =   1.0000E-15 Note: One successful gmin step
Trying gmin =   1.0000E-16 Note: One successful gmin step
Trying gmin =   1.0000E-17 Note: One successful gmin step
Trying gmin =   1.0000E-18 Note: One successful gmin step
Trying gmin =   1.0000E-19 Note: On

Samples generated and saved in AND_3/AND_3_delay.csv. Original files restored.


4. XOR

In [ ]:
directory = 'XOR'
file_name = 'XOR_delay.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
rows_for_delay=df_samples[max(0,num_rows-1):]

generate_delay(parameters,'XOR/45nm_MGK.pm', 'XOR/XOR.net', 'XOR/XOR_delay.csv',2,rows_for_delay)

Samples generated and saved in XOR/XOR_delay.csv. Original files restored.


5. OR2

In [ ]:
directory = 'OR'
file_name = 'OR_2_delay.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
rows_for_delay=df_samples[max(0,num_rows-1):]

generate_delay(parameters,'OR/45nm_MGK.pm', 'OR/OR.net', 'OR/OR_2_delay.csv',2,rows_for_delay)

### 1.3.2 Gates non-essential for C499 Circuit

6. NAND_2

In [ ]:
# directory = 'NAND'
# file_name = 'NAND_2_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'NAND/45nm_MGK.pm', 'NAND/NAND.net', 'NAND/NAND_2_delay.csv',2,rows_for_delay)

7. NAND_3

In [ ]:
# directory = 'NAND_3'
# file_name = 'NAND_3_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'NAND_3/45nm_MGK.pm', 'NAND_3/NAND.net', 'NAND_3/NAND_3_delay.csv',3,rows_for_delay)

8. NAND_4

In [ ]:
# directory = 'NAND_4'
# file_name = 'NAND_4_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'NAND_4/45nm_MGK.pm', 'NAND_4/NAND.net', 'NAND_4/NAND_4_delay.csv',4,rows_for_delay)

9. NOR_2

In [ ]:
# directory = 'NOR'
# file_name = 'NOR_2_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'NOR/45nm_MGK.pm', 'NOR/NOR.net', 'NOR/NOR_2_delay.csv',2,rows_for_delay)

10. NOR_3

In [ ]:
# directory = 'NOR_3'
# file_name = 'NOR_3_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'NOR_3/45nm_MGK.pm', 'NOR_3/NOR.net', 'NOR_3/NOR_3_delay.csv',3,rows_for_delay)

11. NOR_4

In [ ]:
# directory = 'NOR_4'
# file_name = 'NOR_4_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'NOR_4/45nm_MGK.pm', 'NOR_4/NOR.net', 'NOR_4/NOR_4_delay.csv',4,rows_for_delay)

12. AND_4

In [ ]:
# directory = 'AND_4'
# file_name = 'AND_4_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'AND_4/45nm_MGK.pm', 'AND_4/AND.net', 'AND_4/AND_4_delay.csv',4,rows_for_delay)

13. A012

In [ ]:
# directory = 'A012'
# file_name = 'A012_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'A012/45nm_MGK.pm', 'A012/A012.net', 'A012/A012_delay.csv',3,rows_for_delay)

14. A022

In [ ]:
# directory = 'A022'
# file_name = 'A022_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'A022/45nm_MGK.pm', 'A022/A022.net', 'A022/A022_delay.csv',4,rows_for_delay)

15. A031

In [ ]:
# directory = 'A031'
# file_name = 'A031_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'A031/45nm_MGK.pm', 'A031/A031.net', 'A031/A031_delay.csv',4,rows_for_delay)

16. A0112

In [ ]:
# directory = 'A0112'
# file_name = 'A0112_delay.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# rows_for_delay=df_samples[max(0,num_rows-1):]

# generate_delay(parameters,'A0112/45nm_MGK.pm', 'A0112/A0112.net', 'A0112/A0112_delay.csv',4,rows_for_delay)

# 2. Generating files for leakage

## 2.1 Functions

### 2.1.1 Generic numeric functions

In [16]:
import os
import re
import pandas as pd

def round_to_nearest(value, values_list):
    return min(values_list, key=lambda x: abs(x - value))

def round_gate_voltage(value):
    if value < 0.55:
        return 0
    else:
        return 1.1


### 2.1.3 Function to extract voltage and current data from a SPICE simulation output file.

**Inputs:**
1. `filename`: The filename of the SPICE simulation output file.
2. `ratio_list`: A list containing two values representing the W/L ratio for PMOS and NMOS transistors respectively.

**Output:**
1. `df`: A pandas DataFrame containing the extracted data with the following columns:
    - `MOSFET Type`: Type of the MOSFET (either 'PMOS' or 'NMOS').
    - `W_L Ratio`: W/L ratio of the MOSFET.
    - `Drain Voltage`: Voltage at the drain terminal of the MOSFET.
    - `Gate Voltage`: Voltage at the gate terminal of the MOSFET.
    - `Source Voltage`: Voltage at the source terminal of the MOSFET.
    - `Drain Current`: Current flowing through the drain terminal of the MOSFET.
    - `Gate Current`: Current flowing through the gate terminal of the MOSFET.
    - `Source Current`: Current flowing through the source terminal of the MOSFET.
    - `Body Current`: Body current of the MOSFET.

In [17]:
def extract_spice_data(filename, ratio_list):
    text = os.popen(f"ngspice {filename}").read()
    voltage_pattern = re.compile(r"v\((\w+)\) = ([\d\.\-\+e]+)")
    current_pattern = re.compile(r"i\((\w+)\) = ([\d\.\-\+e]+)")

    voltages = []
    currents = []

    for match in re.finditer(voltage_pattern, text):
        voltages.append((match.group(1), float(match.group(2))))

    for match in re.finditer(current_pattern, text):
        currents.append((match.group(1), float(match.group(2))))

    rows = []
    
    round_values = [i/20 for i in range(0, 23)]

    # Determine MOSFET type based on the number of rows in the DataFrame
    mosfet_type_list = ['PMOS'] * (len(voltages) // 6) + ['NMOS'] * (len(voltages) // 6)
    # Calculate W_L ratio for PMOS and NMOS
    pmos_ratio, nmos_ratio = ratio_list

    for i in range(len(voltages) // 3):  # Assuming each MOSFET has 3 voltage values
        index = i * 3
        index_current = i * 4
        mosfet_type = mosfet_type_list[i]
        ratio = pmos_ratio if mosfet_type == 'PMOS' else nmos_ratio
        rows.append({'MOSFET Type': mosfet_type,
                     'W_L Ratio': ratio,
                     'Drain Voltage': round_to_nearest(voltages[index][1], round_values),
                     'Gate Voltage': round_gate_voltage(voltages[index+1][1]), # Assuming gate voltage is constant for all transistors
                     'Source Voltage': round_to_nearest(voltages[index+2][1], round_values),
                     'Drain Current': currents[index_current][1],
                     'Gate Current': currents[index_current+1][1], # Assuming gate current is constant for all transistors
                     'Source Current': currents[index_current+2][1],
                     'Body Current': currents[index_current+3][1]})

    df = pd.DataFrame(rows, columns=['MOSFET Type', 'W_L Ratio', 'Drain Voltage', 'Gate Voltage', 'Source Voltage',
                                     'Drain Current', 'Gate Current', 'Source Current', 'Body Current'])

    return df


### 2.1.4 Function to calculate the total leakage current of a MOSFET based on the true current values, terminal voltages, MOSFET type, and supply voltage.

**Inputs:**
1. `true_values`: A tuple containing the true values of drain current, gate current, source current, and body current.
2. `drain_voltage`: Voltage at the drain terminal.
3. `gate_voltage`: Voltage at the gate terminal.
4. `source_voltage`: Voltage at the source terminal.
5. `mos_type`: Type of the MOSFET ('PMOS' or 'NMOS').
6. `supply`: Supply voltage.

**Output:**
1. `leakage_current`: The calculated total leakage current of the MOSFET.

In [18]:
def calculate_leakage(true_values, drain_voltage, gate_voltage, source_voltage, mos_type,supply):
    # Extracting true currents
    true_drain_current, true_gate_current, true_source_current, true_body_current = true_values
    
    # Checking if the magnitudes of incoming currents match
    
    if mos_type == 'PMOS':
        body_voltage=supply
    else:
        body_voltage=0   
    
    # Calculating outgoing currents based on terminal voltages and direction of current flow
    incoming_currents = []
    if mos_type == 'PMOS':
        # For PMOS, drain is at a lower potential than gate or source
        if (drain_voltage < source_voltage or drain_voltage < body_voltage) and true_drain_current < 0:
            incoming_currents.append(abs(true_drain_current))
        if (gate_voltage < source_voltage or gate_voltage < drain_voltage) and true_gate_current < 0:
            incoming_currents.append(abs(true_gate_current))
        if (source_voltage < drain_voltage or source_voltage < gate_voltage) and true_source_current < 0:
            incoming_currents.append(abs(true_source_current))
        if true_body_current < 0:
            incoming_currents.append(abs(true_body_current))  # Body current always flows out
    elif mos_type == 'NMOS':
        # For NMOS, drain is at a higher potential than gate or source
        if (drain_voltage > source_voltage or drain_voltage > gate_voltage) and true_drain_current < 0:
            incoming_currents.append(abs(true_drain_current))
        if (gate_voltage > source_voltage or gate_voltage > drain_voltage) and true_gate_current < 0:
            incoming_currents.append(abs(true_gate_current))
        if (source_voltage > drain_voltage or source_voltage > gate_voltage) and true_source_current < 0:
            incoming_currents.append(abs(true_source_current))
        if true_body_current < 0:
            incoming_currents.append(abs(true_body_current))  # Body current always flows out
    
    # Calculating leakage current as the sum of outgoing currents
    leakage_current = sum(incoming_currents)
    return leakage_current

### 2.1.5 Function to calculate the total leakage current for all rows in an original list of MOSFET data.

**Inputs:**
1. `original_list`: A list containing MOSFET data where each item is a list with the following format:
    - `original_list[i][0]`: MOSFET type ('PMOS' or 'NMOS').
    - `original_list[i][2]`: Drain voltage.
    - `original_list[i][3]`: Gate voltage.
    - `original_list[i][4]`: Source voltage.
    - `original_list[i][5:9]`: True current values (drain current, gate current, source current, body current).
2. `supply`: Supply voltage.

**Output:**
1. `total_leakage_current`: The sum of all calculated leakage currents for each row in the original list.

In [19]:
def calculate_leakage_for_all_rows(original_list,supply):
    all_leakage_currents = []
    for i in range(len(original_list)):
        leakage_current = calculate_leakage(original_list[i][5:9],
                                            original_list[i][2],
                                            original_list[i][3],
                                            original_list[i][4],
                                            original_list[i][0],supply)
        all_leakage_currents.append(leakage_current)
        
    return sum(all_leakage_currents)

### 2.1.6 Function to generate leakage values and save it in a csv file

Inputs:
1. PTM file (.pm)
2. Circuit Netlist (.net)
3. Number of inputs in the circuit
4. Number of samples to generate
5. CSV file path
6. Parameters

Output:
1. Modified CSV file

In [20]:

import numpy as np
import csv
from itertools import product

def generate_leakage(PM_file, netlist_file, no_of_inputs, csv_file_path, parameters, file_ratio, df_samples):
    # Load file content
    with open(PM_file, 'r') as file:
        original_content = file.read()

    with open(netlist_file, 'r') as file:
        original_content_leakage = file.read()

    # Make a copy of the original content for modification
    modified_content = original_content

    # Replace parameter values in the original file with sampled values
    with open(csv_file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header_row = ['Vin_' + chr(65 + i) for i in range(no_of_inputs)] + ['TEMP', 'pvdd', 'Lmin', 'Wmin', 'toxe_n', 'toxm_n', 'toxref_n', 'toxp_n', 'xj_n', 'ndep_n','toxe_p', 'toxm_p', 'toxref_p', 'toxp_p', 'xj_p', 'ndep_p'] + ['Leakage_power']
        num_rows = sum(1 for _ in open(csv_file_path, 'r'))
        to_skip=(num_rows%(2**no_of_inputs))-1;
        to_skip=(to_skip+(2**no_of_inputs))%2**no_of_inputs
        if num_rows==0:
            writer.writerow(header_row)
            to_skip=0
        skipped=0
        try:
            for i in range(len(df_samples)):
                i+=max(0,num_rows-1)//2**(no_of_inputs)
                lmin = df_samples['lmin'][i]
                wmin = df_samples['wmin'][i]
                temp = df_samples['temp'][i]
                supply = df_samples['supply'][i]

                # Replace parameter values in the original file with sampled values
                for param in parameters:
                    nmos_value=df_samples[param+'_n'][i]
                    pmos_value=df_samples[param+'_p'][i]
                    modified_content = re.sub(rf'\b{param}\s*=\s*([\d.e+-]+)', f'{param} = {pmos_value:.6e}', modified_content, count=2)
                    modified_content = re.sub(rf'\b{param}\s*=\s*([\d.e+-]+)', f'{param} = {nmos_value:.6e}', modified_content, count=1)   
                
                # Save modified content back to the original file
                with open(PM_file, 'w') as file:
                    file.write(modified_content)
                
                

                # Iterate over all possible combinations of input voltages
                for Vin_values in product(range(2), repeat=no_of_inputs):
                    if skipped<to_skip:
                        skipped+=1
                        continue
                    input_values = [(supply if val == 1 else 0) for val in Vin_values]
                    modify_file_leakage(netlist_file, temp, supply, lmin, wmin, *input_values)
                    
                    # Extracting voltage and current values from executing netlist
                    file_df_extracted = extract_spice_data(netlist_file, file_ratio)
                    leakage_currents = calculate_leakage_for_all_rows(file_df_extracted.values.tolist(), supply)

                    row = list(Vin_values) + [temp, supply, lmin * 1e-9, wmin * 1e-9] + [df_samples[param+'_n'][i] for param in parameters]+[df_samples[param+'_p'][i] for param in parameters] + [leakage_currents * (supply + sum(Vin_values))]
                    writer.writerow(row)
                    
        finally:

            # Revert changes back to original values
            with open(PM_file, 'w') as file:
                file.write(original_content)

            with open(netlist_file, 'w') as file:
                file.write(original_content_leakage)

            print(f"Samples generated and saved in '{csv_file_path}'. Original files restored.")


### 2.1.7 Function for and gate

In [21]:
import pandas as pd
from itertools import product

def calculate_and_or_gate_leakage(nand_file_path, inverter_file_path, output_file_path, no_of_inputs):
    # Read the input CSV files
    nand_df = pd.read_csv(nand_file_path)
    inverter_df = pd.read_csv(inverter_file_path)

    # Define the keys for the resulting DataFrame
    nand_parameter_key = [f'Vin_{chr(65+i)}' for i in range(no_of_inputs)] + ['TEMP', 'pvdd', 'Lmin', 'Wmin', 'toxe_n', 'toxm_n', 'toxref_n', 'toxp_n', 'xj_n', 'ndep_n', 'toxe_p', 'toxm_p', 'toxref_p', 'toxp_p', 'xj_p', 'ndep_p']
    and_df = pd.DataFrame({key: [] for key in nand_parameter_key + ['Leakage_power']})

    # Define the key for leakage power
    leakage_key = 'Leakage_power'

    # Iterate over the rows of NAND_df
    cur_pointer=0
    for index, nand_row in nand_df.iterrows():
        # Get the corresponding inverter input for the current NAND input combination
        input_combination = nand_row[[f'Vin_{chr(65+i)}' for i in range(no_of_inputs)]].tolist()
        inverter_input = 0 if all(bit == 1 for bit in input_combination) else 1
        
        
        # Sum the leakage power of the NAND gate and the matching inverter
        row_no=cur_pointer+(inverter_input==1)
        total_leakage_power = nand_row[leakage_key] + inverter_df.loc[row_no,leakage_key]

        # Append the values to the DataFrame
        and_df = pd.concat([and_df, pd.DataFrame({**dict(nand_row), leakage_key: total_leakage_power}, index=[0])], ignore_index=True)
        
        if (index+1)%((2**no_of_inputs))==0:
            cur_pointer += 2
        

    # Save the resulting DataFrame to a CSV file
    and_df.to_csv(output_file_path, index=False)
    
    print(f"Samples generated and saved in '{output_file_path}'. Original files restored.")



## 2.2 Defining Parameters to search for

In [22]:
# List of parameters to search for
parameters = [
    'toxe', 'toxm', 'toxref','toxp', 'xj', 'ndep'
]

## 2.3 Generating Leakage matrices for all files

### 2.3.1 Gates essential for C499 Circuit

1. INVERTER

In [ ]:
directory = 'INVERTER'
file_name = 'INVERTER_leakage.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
no_of_inputs = 1  # Number of inputs in the circuit
rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

generate_leakage('INVERTER/45nm_MGK.pm', 'INVERTER/INVERTER_leakage.net', no_of_inputs, 'INVERTER/INVERTER_leakage.csv',parameters,[2,1],rows_for_leakage)

2. NAND_2

In [ ]:
directory = 'NAND'
file_name = 'NAND_2_leakage.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
no_of_inputs = 2  # Number of inputs in the circuit
rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

generate_leakage('NAND/45nm_MGK.pm', 'NAND/NAND_leakage.net', no_of_inputs, 'NAND/NAND_2_leakage.csv', parameters, [2,2],rows_for_leakage)


3. NAND_3

In [ ]:
directory = 'NAND_3'
file_name = 'NAND_3_leakage.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
no_of_inputs = 3  # Number of inputs in the circuit
rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

generate_leakage('NAND_3/45nm_MGK.pm', 'NAND_3/NAND_leakage.net', no_of_inputs, 'NAND_3/NAND_3_leakage.csv',parameters,[2,3],rows_for_leakage)

4. NOR_2

In [ ]:
directory = 'NOR'
file_name = 'NOR_2_leakage.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
# NOR_2
no_of_inputs = 2  # Number of inputs in the circuit
rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

generate_leakage('NOR/45nm_HP.pm', 'NOR/NOR_leakage.net', no_of_inputs, 'NOR/NOR_2_leakage.csv', parameters, [4, 1],rows_for_leakage)

5. XOR

In [ ]:
directory = 'XOR'
file_name = 'XOR_leakage.csv'
file_path = os.path.join(directory, file_name)

# Check if the file already exists
if not os.path.exists(file_path):
    # Write some data to the file
    data = ""
    with open(file_path, 'w') as file:
        file.write(data)

# Open the CSV file and count the rows
num_rows = sum(1 for _ in open(file_path, 'r'))
no_of_inputs = 2  # Number of inputs in the circuit
rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

generate_leakage('XOR/45nm_MGK.pm', 'XOR/XOR_leakage.net', no_of_inputs, 'XOR/XOR_leakage.csv',parameters,[2,1],rows_for_leakage)

6. AND_2

In [ ]:
# For AND2
calculate_and_or_gate_leakage('NAND/NAND_2_leakage.csv', 'INVERTER/INVERTER_leakage.csv', 'AND/AND_2_leakage.csv',2)


7. AND_3

In [ ]:
# For AND3
calculate_and_or_gate_leakage('NAND_3/NAND_3_leakage.csv', 'INVERTER/INVERTER_leakage.csv', 'AND_3/AND_3_leakage.csv',3)


8. OR_2

In [ ]:
# For AND2
calculate_and_or_gate_leakage('NOR/NOR_2_leakage.csv', 'INVERTER/INVERTER_leakage.csv', 'OR/OR_2_leakage.csv',2)


### 2.3.2 Gates non-essential for C499 Circuit

9. NAND_4

In [ ]:
# directory = 'NAND_4'
# file_name = 'NAND_4_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# no_of_inputs = 4  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('NAND_4/45nm_MGK.pm', 'NAND_4/NAND_leakage.net', no_of_inputs, 'NAND_4/NAND_4_leakage.csv',parameters,[2,4],rows_for_leakage)

10. AND_4

In [ ]:
# # For AND4
# calculate_and_or_gate_leakage('NAND_4/NAND_4_leakage.csv', 'INVERTER/INVERTER_leakage.csv', 'AND_4/AND_4_leakage.csv',4)


11. NOR_3

In [ ]:
# directory = 'NOR_3'
# file_name = 'NOR_3_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# # NOR_3
# no_of_inputs = 3  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('NOR_3/45nm_MGK.pm', 'NOR_3/NOR_leakage.net', no_of_inputs, 'NOR_3/NOR_3_leakage.csv', parameters, [6, 1],rows_for_leakage)


12. NOR_4

In [ ]:
# directory = 'NOR_4'
# file_name = 'NOR_4_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# no_of_inputs = 4  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('NOR_4/45nm_MGK.pm', 'NOR_4/NOR_leakage.net', no_of_inputs, 'NOR_4/NOR_4_leakage.csv',parameters,[8,1],rows_for_leakage)

13. A012

In [ ]:
# directory = 'A012'
# file_name = 'A012_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# no_of_inputs = 3  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('A012/45nm_MGK.pm', 'A012/A012_leakage.net', no_of_inputs, 'A012/A012_leakage.csv',parameters,[2,1],rows_for_leakage)

14. A022

In [ ]:
# directory = 'A022'
# file_name = 'A022_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# no_of_inputs = 4  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('A022/45nm_MGK.pm', 'A022/A022_leakage.net', no_of_inputs, 'A022/A022_leakage.csv',parameters,[2,1],rows_for_leakage)

15. A031

In [ ]:
# directory = 'A031'
# file_name = 'A031_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# no_of_inputs = 4  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('A031/45nm_MGK.pm', 'A031/A031_leakage.net', no_of_inputs, 'A031/A031_leakage.csv',parameters,[4,3],rows_for_leakage)

16. A0112

In [ ]:
# directory = 'A0112'
# file_name = 'A0112_leakage.csv'
# file_path = os.path.join(directory, file_name)

# # Check if the file already exists
# if not os.path.exists(file_path):
#     # Write some data to the file
#     data = ""
#     with open(file_path, 'w') as file:
#         file.write(data)

# # Open the CSV file and count the rows
# num_rows = sum(1 for _ in open(file_path, 'r'))
# no_of_inputs = 4  # Number of inputs in the circuit
# rows_for_leakage=df_samples[max(0,num_rows-1)//(2**no_of_inputs):]

# generate_leakage('A0112/45nm_MGK.pm', 'A0112/A0112_leakage.net', no_of_inputs, 'A0112/A0112_leakage.csv',parameters,[3,1],rows_for_leakage)